In [4]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
data_root='/content/drive/My Drive/ChatBot'

Mounted at /content/drive


In [5]:
# Importing relevant libraries
import json
import string
import random
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

nltk.download("punkt")
nltk.download("wordnet")

#Loading the dataset: intents.json

data_file = open(data_root + '/intents.json').read()
data = json.loads(data_file)

# Creating data_x and Data_Y
words = []  # for model/ vocabulary for patterns
classes = []  # for bow model/ vocabulary for tags
data_X = []  # for storing each pattern
data_Y = []  # for storing tags corresponding to each pattern in data_x

# Iterating over all the intents
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)  # tokenize each pattern
        words.extend(tokens)  # and append tokens to words
        data_X.append(pattern)  # appending pattern to data_X
        data_Y.append(intent["tag"])  # appending the associated tag to each pattern

        # Adding the tag to the classes if it's not there already
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

# Initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()

# Lemmatize all the words in the vocab and convert them to lowercase
# if the words don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]

# Sorting the vocab and classes in alphabetical order and taking the set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))

# Text to numbers
training = []
out_empty = [0] * len(classes)

# Creating the bag of words model
for idx, doc in enumerate(data_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)

    # Mark the index of class that the current pattern is associated with
    output_row = list(out_empty)
    output_row[classes.index(data_Y[idx])] = 1

    # Add the one hot encoded BoW and associated classes to training
    training.append([bow, output_row])

# Shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)

# Split the features and target labels
train_X = np.array(list(training[:, 0]))
train_Y = np.array(list(training[:, 1]))

# The neural network model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_X[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_Y[0]), activation="softmax"))
adam = tf.keras.optimizers.legacy.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_Y, epochs=150, verbose=1)

# Preprocessing the inputs
def clean_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

def bag_of_words(text, vocab):
    tokens = clean_text(text)
    bow = [0] * len(vocab)
    for w in tokens:
        for idx, word in enumerate(vocab):
            if word == w:  # Corrected comparison
                bow[idx] = 1
    return np.array(bow)

def pred_class(text, vocab, labels):
    bow = bag_of_words(text, vocab)
    result = model.predict(np.array([bow]))[0]
    thresh = 0.5
    y_pred = [[indx, res] for indx, res in enumerate(result) if res > thresh]
    y_pred.sort(key=lambda x: x[1], reverse=True)  # Sorting by values of probability in decreasing order
    return_list = []
    for r in y_pred:
        return_list.append(labels[r[0]])
    return return_list

def get_response(intents_list, intents_json):
    if len(intents_list) == 0:
        result = random.choice(intents_json["intents"][1]["responses"])  # Default response for no matching intent
    else:
        tag = intents_list[0]
        list_of_intents = intents_json["intents"]
        for i in list_of_intents:
            if i["tag"] == tag:
                result = random.choice(i["responses"])
                break
    return result


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               6784      
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 11)                715       
                                                                 
Total params: 15755 (61.54 KB)
Trainable params: 15755 (61.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/150
2

In [ ]:

# Interacting with the chatbot
print("Type '0' if you don't want to chat with our ChatBot.")
while True:
    message = input("")
    if message == "0":
        break
    intents = pred_class(message, words, classes)
    result = get_response(intents, data)
    print(result)


Type '0' if you don't want to chat with our ChatBot.
hey
1/1 [==============================] - 0s 31ms/step
Hello! How can I assist you today?
apply for loan
1/1 [==============================] - 0s 22ms/step
Interested in a loan? Let's discuss your options.
file a complaint
1/1 [==============================] - 0s 21ms/step
I'm sorry to hear that. Please provide details about your complaint, and I'll escalate it to our support team.
show recent transactions
1/1 [==============================] - 0s 22ms/step
Sure! I can provide you with your recent transaction history.
